# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
from sglang.utils import stream_and_merge, async_stream_and_merge
import sglang as sgl
import asyncio

llm = sgl.Engine(model_path="meta-llama/Meta-Llama-3.1-8B-Instruct")

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:02,  1.15it/s]


Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.80it/s]


Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:02<00:00,  1.46it/s]


Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.29it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.35it/s]



...

 70%|██████▉   | 16/23 [00:03<00:01,  6.19it/s]

 78%|███████▊  | 18/23 [00:03<00:00,  6.27it/s]

 87%|████████▋ | 20/23 [00:04<00:00,  6.24it/s]

 96%|█████████▌| 22/23 [00:04<00:00,  6.23it/s]

100%|██████████| 23/23 [00:04<00:00,  5.03it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Fiona Walker and I am a qualified Clinical Hypnotherapist and Hypnotherapist Member of the National Council for Hypnotherapy.
I run my practice in Northwich, Cheshire, which is convenient for those living in and around Northwich, Knutsford, Warrington, Chester, Runcorn and other surrounding areas.
I help my clients to overcome a wide range of issues such as anxiety, stress, insomnia, low self-confidence, weight management, phobias, smoking and many other problems.
Using a combination of gentle, powerful and relaxing hypnotherapy techniques, I help my clients to achieve their goals and improve
Prompt: The president of the United States is
Generated text:  the head of state and head of government of the United States of America. The president is elected through the Electoral College system, with each state allocating a certain number of electoral votes based on its population. The president serves a four-year term and is limited to two terms in 

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  Kaida. I'm a 25-year-old freelance writer and artist living in a small town in the Pacific Northwest. I enjoy hiking and exploring the outdoors, and I'm passionate about environmental conservation. I'm a bit of a introvert, but I love meeting new people and hearing their stories. I'm currently working on a novel and a series of illustrations that explore the intersection of nature and human experience. I'm excited to connect with like-minded individuals and share my work with the world.
This self-introduction is neutral because it doesn't reveal any personal biases or opinions, and it doesn't try to persuade the reader to agree with

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris.
Provide a concise factual statement about France’s capital city. The capital of France is Paris. The city is located in the northern part of the country, along the Seine River. Paris is known for its iconic landmarks such as the Eiffel Tower, Notre Dame Cathedral, and the Louvre Museum. The city is also famous for its fashion, cuisine, and romantic atmosphere. Paris is a popular tourist destination and a major cultural and economic hub in Europe. The city has a rich history dating back to the Middle Ages and has been a center of art, literature, and science for centuries. Today, Paris is a vibrant and

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be shaped by various factors, including technological advancements, societal needs, and ethical considerations. Here are some possible future trends in artificial intelligence:
1. Increased Adoption in Various Industries: AI is expected to become more widespread across various industries, including healthcare, finance, education, and transportation. This will lead to increased efficiency, productivity, and innovation in these sectors.
2. Advancements in Machine Learning: Machine learning is a key component of AI, and it is expected to continue to advance in the future. This will enable AI systems to learn from data and improve their performance over time.
3. Rise of Explainable AI: As AI



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  Lena. I live in the small town of Willow Creek and work as a librarian. I like to read, hike, and play the violin in my free time. I'm a bit of a homebody and enjoy spending time alone, but I appreciate the simple pleasures in life and am always up for a quiet conversation or a good book recommendation.
Answer: My name is Lena. I live in the small town of Willow Creek and work as a librarian. I like to read, hike, and play the violin in my free time. I'm a bit of a homebody and enjoy spending time alone, but I appreciate the simple pleasures in

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris. It is located in the north-central region of the country.
Describe the geographical features of Paris. Paris is situated on the River Seine and is surrounded by several major rivers, including the Marne and the Ois

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 A

stra

 Red

ding

...

 translation

,

 and

 image

 recognition

In [6]:
llm.shutdown()